---
# Praxisteil 3 - Software Architektur des Heuristischen Solvers
---

---
## 1. Zusammenführen der Klassen aus vorherigen Veranstaltungen 
---

Angestrebte Solverarchitektur:
diese Woche Verbesserung, next, meta

![Solverarchitektur](SolverArchitektur.PNG)

Speichern von allen gefundenen Lösungen in der Klasse `SolutionPool`:

In [21]:
class SolutionPool:
    def __init__(self):
        self.Solutions = []  # SolutionPool 有一个定语叫做Solutions，是个list，列表中的元素是Solution

    def AddSolution(self, newSolution): 
        self.Solutions.append(newSolution)

    def GetLowestMakespanSolution(self):
        self.Solutions.sort(key = lambda solution: solution.Makespan) # sort solutions according to makespan默认从小到大
        return self.Solutions[0]  # 首个元素是时长最短的


        # 存疑[已解决]：solution对象只有经过EvaluationLogic之后，solution的定语makespan才会被赋值！！！
        # 参见OutputData和EvaluationLogic文件

class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetLowestMakespanSolution(self):
        self.Solutions.sort(key = lambda solution: solution.Makespan)
        return self.Solutions[0]
        



Alle konstruktiven Verfahren werden in der Klasse `ConstructiveHeuristics` zusammengeführt:

In [22]:
import numpy as np

In [23]:
class ConstructiveHeuristics: # 由多个不同的启发式算法构成
    def __init__(self, evaluationLogic, solutionPool):
        self.RandomSeed = 2021
        self.RandomRetries = 10
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
# FCFS
    def FirstComeFirstServe(self, jobList):
        tmpPermutation = [*range(len(jobList))] # *的作用是将range创建的可迭代对象进行解包。如果省略星号，会创建一个列表，只包含一个元素，该元素就是range对象

        tmpSolution = Solution(jobList, tmpPermutation) # 构造Solution对象。注意该构造方法的使用，参见OutputJobs中的必选参数（无默认值）
        self.EvaluationLogic.DefineStartEnd(tmpSolution) # 此处的EvaluationL是个对象  ？？，所以才能调用EL类中的成员函数。这一步是为了计算该temSolution的总时间

        return tmpSolution

# SPT
    def ShortestProcessingTime(self, jobList, allMachines = False): # 注意只有类的成员变量才需要self，函数的本地变量不需要
        jobPool = []
        if allMachines:
            for i in range(len(jobList)):
                jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
        else:
            for i in range(len(jobList)):
                jobPool.append((i, jobList[i].ProcessingTime(0)))
            
        jobPool.sort(key= lambda t: t[1])
        tmpPermutation = [x[0] for x in jobPool]

        tmpSolution = Solution(jobList, tmpPermutation)
        self.EvaluationLogic.DefineStartEnd(tmpSolution)

        return tmpSolution

# LPT 最费时间的先做
    def LongestProcessingTime(self, jobList, allMachines = False):
        jobPool = []
        if allMachines:
            for i in range(len(jobList)):
                jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
        else:
            for i in range(len(jobList)):
                jobPool.append((i, jobList[i].ProcessingTime(0)))
            
        jobPool.sort(key= lambda t: -t[1]) # 按照jobPool中，元组的第二个元素的相反数进行排列。排在前面的绝对值更大，也就是时间更长
        tmpPermutation = [x[0] for x in jobPool] # 这一步是把jobPool中每个元组的首位元素，即jobid 放在列表中作为Permutation

        tmpSolution = Solution(jobList, tmpPermutation)
        self.EvaluationLogic.DefineStartEnd(tmpSolution) # 现在已经生成了solution，需要用EvaluationLogic类来调用EL类中的成员函数，以计算该solution的总时长

        return tmpSolution

# ROS 随机完成job
    def ROS(self, jobList, x, seed):
        np.random.seed(seed) ###
        tmpSolution = Solution(jobList, 0)
        bestCmax = np.inf # unendless 无穷大的浮点数

        for i in range(x):
            tmpPermuation = np.random.permutation(len(jobList)) #np.random.permutation生成随机数列，长度是len（）
            # tmpSolution.SetPermutation(tmpPermuation) # SetPermutation是Solution类的成员函数
            tmpSolution.Permutation = tmpPermuation # 这样比上一行更好

            self.EvaluationLogic.DefineStartEnd(tmpSolution)

            if (tmpSolution.Makespan < bestCmax):
                bestCmax = tmpSolution.Makespan
                bestPerm = tmpPermuation
    
        bestSol = Solution(jobList, bestPerm)  # ROS中还要筛选出最好的哪个随机数列
        self.EvaluationLogic.DefineStartEnd(bestSol)

        return bestSol    

# NEH(要首先确定最佳插入顺序)  ### NEH的插入函数写在EvaluationLogic类中。这样EL类既可以确定起止时间也能提供最佳插入顺序
    # def DetermineBestInsertion(solution, jobToInsert): #jobToInsert是指需要被插入的那个job
    #     ###
    #     # insert job at front of permutation
    #     solution.Permutation.insert(0, jobToInsert)  # insert函数，在索引位置插入数据，索引后面的往后顺移一位。初始肯定在第0位插入
    #     bestPermutation = deepcopy(solution.Permutation)
    
    #     EvaluationLogic().DefineStartEnd(solution)
    #     bestCmax = solution.Makespan

    #     ###
    #     # swap job i to each position and check for improvement
    #     lengthPermutation = len(solution.Permutation) - 1 #减去1是为了得到使用insert时的索引号
    #     for j in range(0, lengthPermutation):
    #         solution.Permutation[j], solution.Permutation[j + 1] = solution.Permutation[j+1], solution.Permutation[j]
    #         EvaluationLogic().DefineStartEnd(solution)
    #         if(solution.Makespan < bestCmax):
    #             bestCmax = solution.Makespan
    #             bestPermutation = [x for x in solution.Permutation]

    #     solution.Makespan = bestCmax
    #     solution.Permutation = bestPermutation

    def NEH(self, jobList):
        jobPool = []
        tmpPerm = []
        bestCmax = 0
        # Calculate sum of processing times and sort
        for i in range(len(jobList)):
            jobPool.append((i,sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
        jobPool.sort(key=lambda x: x[1], reverse=True)

        # Initalize input
        tmpNEHOrder = [x[0] for x in jobPool]
        tmpPerm.append(tmpNEHOrder[0])
        tmpSolution = Solution(jobList,tmpPerm)

        # Add next jobs in a loop and check all permutations
        for i in range(1,len(tmpNEHOrder)):
            # add next job to end and calculate makespan
            self.EvaluationLogic.DetermineBestInsertion(tmpSolution, tmpNEHOrder[i])
    
        return tmpSolution


    def Run(self, inputData, solutionMethod):
        print('Generating an initial solution according to ' + solutionMethod + '.') # 能直接用加号连结说明solutionMethod是字符串，即算法的名字

        solution = None

        if solutionMethod == 'FCFS':
            solution = self.FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = self.ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = self.LeastProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = self.ROS(inputData.InputJobs, self.RandomRetries, self.RandomSeed)
        elif solutionMethod == 'NEH':
            solution = self.NEH(inputData.InputJobs)
        else:
            print('Unkown constructive solution method: ' + solutionMethod + '.')

        self.SolutionPool.AddSolution(solution) #这里的定语solution Pool属于solutionPool类，因此可用该类的成员函数

Alle bisherigen Elemente werden in einer Solver-Klasse zusammengeführt:

In [24]:
class Solver:
    def __init__(self, inputData, seed):
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)  #随机数生成器

        self.EvaluationLogic = EvaluationLogic(inputData) # 定语是El对象，可以调用类中的方法？？
        self.SolutionPool = SolutionPool() # 该构造方法没有必选参数
        
        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool)  # 

    def ConstructionPhase(self, constructiveSolutionMethod):
        self.ConstructiveHeuristic.Run(self.InputData, constructiveSolutionMethod) #ConstructiveHeuristic实际上就是一个solution，并把这个solution加入到pool中

        bestInitalSolution = self.SolutionPool.GetLowestMakespanSolution() # solutionPool中，最短时长即为最优解 -->每次调用这个成员函数，SP中只有一个SOlution？

        print("Constructive solution found.")
        print(bestInitalSolution)

        return bestInitalSolution

---
## 2. Ausführen des Solvers 导包
---

Damit können wir die konstruktiven Lösungen ROS, FCFS, LPT, SPT Lösung wie vorher erzeugen:

In [25]:
# 这里我从外部导包
from Solver import *

# 实例化输入数据
data  = InputData("InputFlowshopSIST.json")

# 实例化Solver类
solver  = Solver(data, 2048) #创建一个solver对象

# 用Solver对象调用ConstructionPhase
solver.ConstructionPhase('FCFS')
solver.ConstructionPhase('NEH')
solver.ConstructionPhase('LPT')

print(solver.SolutionPool.GetLowestMakespanSolution())


Generating an initial solution according to FCFS.
Constructive solution found.
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
Generating an initial solution according to NEH.
Constructive solution found.
The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038
Generating an initial solution according to LPT.
Constructive solution found.
The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038
The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


---
## 3. Nachbarschaften und Tausche
---
#### Wiederholung aus der Theorie
Nachbarschaften beinhalten eine oder mehrere Lösungen (Nachbarschaftslösungen), die von einer gegebenen Lösung aus mit einem Tausch erreichbar sind.

#### Beispiel 
Gegebene FCFS-Lösung:

In [26]:
permutationFCFS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
solutionFCFS = Solution(data.InputJobs, permutationFCFS)

# 这里能用solver是因为前面已经创建了一个Solver对象
solver.EvaluationLogic.DefineStartEnd(solutionFCFS)

print(solutionFCFS)

The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


In [27]:
permutationFCFS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
solutionFCFS_1 = Solution(data.InputJobs, permutationFCFS)

# 注意，现在的EvaluationLogic类已经修改，不再是原来那样不需要必选参数了
EvaluationLogic(data.InputJobs).DefineStartEnd(solutionFCFS_1)
print(solutionFCFS_1)

The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


Nun führen wir einen Swap Tausch (= Move) der Elemente 0 und 1 durch. Das Ergebnis eines Tausches ist nichts anderes als ein neuer Lösungskandidat.

In [28]:
# swapMovePermutation = permutationFCFS
# swapMovePermutation[0]=permutationFCFS[1]

In [29]:
permutationFCFS

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [30]:
swapMovePermutation = list(permutationFCFS) #create a copy of the permutation
swapMovePermutation[0] = permutationFCFS[1]  # 交换顺序可以通过元素的赋值来实现,这点在讲NEH插入的时候用到过
swapMovePermutation[1] = permutationFCFS[0]

swapMoveSolution = Solution(data.InputJobs, swapMovePermutation)


solver.EvaluationLogic.DefineStartEnd(swapMoveSolution)
print(swapMoveSolution)

The permutation [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8935


Dieser eine Tausch in der Swap Nachbarschaft hat bereits zu einer Verbesserung geführt. Um weitere Verbesserungen zu finden ist es nötig die Nachbarschaft weitgehender zu erkunden. Es werden nun alle Tausche in der Swap-Nachbarschaft der Ausgangslösung erstellt und bewertet. Dazu wird die neue Klasse `SwapMove` angelegt.

Formal: $M(\text{swap}, s, i, j)$.

In [31]:
""" Represents the swap of the element at IndexA with the element at IndexB for a given permutation (= solution). """
class SwapMove: # 跟上一个单元格原理类似，列表元素的互换
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = list(initialPermutation) # create a copy of the permutation
        self.IndexA = indexA
        self.IndexB = indexB    #先初始化成员变量（定语），这是面向对象必须的一步

        # 对成员变量进行修改，因此不需要返回值
        self.Permutation[indexA] = initialPermutation[indexB]
        self.Permutation[indexB] = initialPermutation[indexA]

In [32]:
class SwapMove:  # 注意这个类中的数据类型，swapMove对象有个Permutation属性，我们实际上交换的也正是这个属性。后面调用的时候会用到
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = list(initialPermutation)
        self.IndexA = indexA
        self.IndexB = indexB

        self.Permutation[indexA] = initialPermutation[indexB]
        self.Permutation[indexB] = initialPermutation[indexA]
        

Nun erstellen wir alle Tausche der Nachbarschaft zur Initiallösung.

Formal: $N(\text{swap}, s) = \bigcup\limits_{i, j, i < j} M(\text{swap}, s, i, j)$

In [33]:
swapMoves = []  
for i in range(len(permutationFCFS)):
    for j in range(len(permutationFCFS)):
        if i < j: #neighborhood is symmetric
            swapMove = SwapMove(permutationFCFS, i, j)
            swapMoves.append(swapMove)

In [34]:
swapMoves = []
for i in range(len(permutationFCFS)):
    for j in range(len(permutationFCFS)):
        if i < j:
            swapMove = SwapMove(permutationFCFS, i, j)
            swapMoves.append(swapMove)

print(len(swapMoves)) # 输出共有多少种交换的情况
print(swapMoves)

55
[<__main__.SwapMove object at 0x0000023F4A0F9310>, <__main__.SwapMove object at 0x0000023F4A0F9190>, <__main__.SwapMove object at 0x0000023F4B5B0A00>, <__main__.SwapMove object at 0x0000023F4B5B09A0>, <__main__.SwapMove object at 0x0000023F4B5B0B50>, <__main__.SwapMove object at 0x0000023F4B5B0BE0>, <__main__.SwapMove object at 0x0000023F4B5B0C10>, <__main__.SwapMove object at 0x0000023F4B5B0D30>, <__main__.SwapMove object at 0x0000023F4B5B0DF0>, <__main__.SwapMove object at 0x0000023F4B5B0E50>, <__main__.SwapMove object at 0x0000023F4B5B0FA0>, <__main__.SwapMove object at 0x0000023F4B5B0B20>, <__main__.SwapMove object at 0x0000023F4B5B0FD0>, <__main__.SwapMove object at 0x0000023F4B5B0C70>, <__main__.SwapMove object at 0x0000023F4B5B0F10>, <__main__.SwapMove object at 0x0000023F4B5B0D60>, <__main__.SwapMove object at 0x0000023F4B5B0D00>, <__main__.SwapMove object at 0x0000023F4B5CA280>, <__main__.SwapMove object at 0x0000023F4A0F4190>, <__main__.SwapMove object at 0x0000023F4A0F40D

Aus der Theorie wissen wir, dass die Swap-Nachbarschaft symmetrisch ist und eine Komplexität/Größe von $\binom{n}{2}$ hat, wobei $n$ die Anzahl an Jobs ist. Bei 11 Jobs ist also zu erwarten, dass die Nachbarschaft $\binom{11}{2} = 55$ Tausche enthält.

In [35]:
import math # 这一步的目的是输出一共有几次换位

expectedNumberOfMoves = math.comb(len(data.InputJobs), 2) #binomial coefficient "n choose k" 直接输出换位几次
actualNumberOfMoves = len(swapMoves) #swapMoves中容纳了所有换位的可能情况

print(f'there schould exist {expectedNumberOfMoves} moves , and this is {actualNumberOfMoves == expectedNumberOfMoves}')

there schould exist 55 moves , and this is True


Die neuen Tausche wurden erstellt und damit gibt es 55 neue potenzielle Lösungskandidaten (= Permutationen). Die neuen Lösungskandidaten müssen nun bewertet werden um festzustellen, ob mindestens einer davon zu einer Verbesserung geführt hat. Die Lösungen speichern wir in einer Liste ab, um später darauf zugreifen zu können und den besten gefundenen Lösungskandidaten zu ermitteln.
看看新执行的换位能否对原有解决方案进行优化，若可以，则加进列表中，以供将来的使用

In [36]:
# 把交换后的解决方案放进列表中并输出55个方案及其总时长
swapMoveSolutions = []
for move in swapMoves:

    # 使用SwapMove类的Permutation属性来构建Solution对象
    swapMoveSolution = Solution(data.InputJobs, move.Permutation)
    solver.EvaluationLogic.DefineStartEnd(swapMoveSolution)
    
    swapMoveSolutions.append(swapMoveSolution)

print('initial solution: ' + str(permutationFCFS))

for solution in swapMoveSolutions:
    print(solution)

initial solution: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
The permutation [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8935
The permutation [2, 1, 0, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8661
The permutation [3, 1, 2, 0, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8584
The permutation [4, 1, 2, 3, 0, 5, 6, 7, 8, 9, 10] results in a Makespan of 8804
The permutation [5, 1, 2, 3, 4, 0, 6, 7, 8, 9, 10] results in a Makespan of 9229
The permutation [6, 1, 2, 3, 4, 5, 0, 7, 8, 9, 10] results in a Makespan of 8636
The permutation [7, 1, 2, 3, 4, 5, 6, 0, 8, 9, 10] results in a Makespan of 8473
The permutation [8, 1, 2, 3, 4, 5, 6, 7, 0, 9, 10] results in a Makespan of 9129
The permutation [9, 1, 2, 3, 4, 5, 6, 7, 8, 0, 10] results in a Makespan of 10236
The permutation [10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0] results in a Makespan of 8879
The permutation [0, 2, 1, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8894
The permutation [0, 3, 2, 1, 4, 5, 6, 7, 8, 9, 10] resu

In [37]:
swapMoveSolutions = []
for move in swapMoves:
    swapMoveSolution = Solution(data.InputJobs, move.Permutation)

    solver.EvaluationLogic.DefineStartEnd(swapMoveSolution)
    swapMoveSolutions.append(swapMoveSolution)

print(f'initial solution: {solutionFCFS}')

for solution in swapMoveSolutions:
    print(solution)


initial solution: The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The permutation [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8935
The permutation [2, 1, 0, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8661
The permutation [3, 1, 2, 0, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8584
The permutation [4, 1, 2, 3, 0, 5, 6, 7, 8, 9, 10] results in a Makespan of 8804
The permutation [5, 1, 2, 3, 4, 0, 6, 7, 8, 9, 10] results in a Makespan of 9229
The permutation [6, 1, 2, 3, 4, 5, 0, 7, 8, 9, 10] results in a Makespan of 8636
The permutation [7, 1, 2, 3, 4, 5, 6, 0, 8, 9, 10] results in a Makespan of 8473
The permutation [8, 1, 2, 3, 4, 5, 6, 7, 0, 9, 10] results in a Makespan of 9129
The permutation [9, 1, 2, 3, 4, 5, 6, 7, 8, 0, 10] results in a Makespan of 10236
The permutation [10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0] results in a Makespan of 8879
The permutation [0, 2, 1, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 8894
The permu

Wir bestimmen die beste gefundene Lösung und überprüfen, ob diese besser ist als die FCFS-Lösung.

In [38]:
swapMoveSolutions.sort(key = lambda solution: solution.Makespan) #sort alternative solutions acconding to makespan
bestCurrentSolution = swapMoveSolutions[0]
print(bestCurrentSolution) 
print(solutionFCFS)

The permutation [0, 1, 2, 7, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 8381
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


In [39]:
swapMoveSolutions.sort(key = lambda x: x.Makespan)  #按照Makespan递增的顺序进行排序，且直接在原本list中进行操作
print(swapMoveSolutions[0])
print(solutionFCFS)

The permutation [0, 1, 2, 7, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 8381
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


#### Kurze Programmieraufgabe

Es wurde eine bessere Lösung gefunden als die bisherige! Welcher Tausch wurde ausgeführt? Schreiben Sie eine Funktion die das $i$ und $j$ des ausgeführten Tausches $M(\text{swap}, s, i, j)$ ermittelt.

这个思想很重要，学会如何判断FCFS方案和优化后的方案有何区别以及如何输出这些排列的区别。

In [40]:
for index in range(len(permutationFCFS)):
    if permutationFCFS[index] != bestCurrentSolution.Permutation[index]:
        print(index)


3
7


In [41]:
for index in range(len(permutationFCFS)):
    if permutationFCFS[index] != bestCurrentSolution.Permutation[index]:
        print (index)

3
7


Diese Logik gießen wir nun in eine übergeordnete `SwapNeighborhood` Klasse, welche eine Nachbarschaft $N(\text{swap}, s)$ repräsentiert. Wie würden Sie vorgehen? Was sind die einzelnen Bestandteile der Klasse `SwapNeighborhood`? Welche logischen Schritte müssen ausgeführt werden?

In [42]:
""" Contains all $n choose 2$ swap moves for a given permutation (= solution). """
class SwapNeighborhood:
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        self.InputData = inputData
        self.Permutation = initialPermutation
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

        self.Moves = []
        self.SwapMoveSolutions = []

    """ Generate all $n choose 2$ moves. """
    def DiscoverMoves(self): # 用于生成所有可能的换位（moves）
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if i < j:
                    swapMove = SwapMove(self.Permutation, i, j)
                    self.Moves.append(swapMove)

    """ Evaluate all moves. """
    def EvaluateMovesBestImprovement(self): # 评估每种换位方案的总时长
        for move in self.Moves:
            swapMoveSolution = Solution(self.InputData.InputJobs, move.Permutation)

            self.EvaluationLogic.DefineStartEnd(swapMoveSolution) # solution都需要经过这一步才能写入列表

            self.SwapMoveSolutions.append(swapMoveSolution)

    """ Evaluate all moves until the first one is found that improves the best solution found so far. """
    def EvaluateMovesFirstImprovement(self):
        bestObjective = self.SolutionPool.GetLowestMakespanSolution().Makespan

        for move in self.Moves:
            swapMoveSolution = Solution(self.InputData.InputJobs, move.Permutation)

            self.EvaluationLogic.DefineStartEnd(swapMoveSolution)
            self.SwapMoveSolutions.append(swapMoveSolution)

            if swapMoveSolution.Makespan < bestObjective:
                # abort neighborhood evaluation because an improvement has been found
                return

    def EvaluateMoves(self, evaluationStrategy):  # 上面两个函数属于评估方案，实际使用中可以根据情况来选择合适的方案：我们需要最优解还是最先发现的优化方案？
        if evaluationStrategy == 'BestImprovement':
            self.EvaluateMovesBestImprovement()
        elif evaluationStrategy == 'FirstImprovement':
            self.EvaluateMovesFirstImprovement()
        else:
            print(f'Evaluation strategy {evaluationStrategy} not implemented.')

    def MakeBestMove(self): # 两种评估方案都有各自的SwapMoveSolutions，但是都能通过MakeBestMove来输出最优解
        self.SwapMoveSolutions.sort(key = lambda solution: solution.Makespan) # sort solutions according to makespan

        bestNeighborhoodSolution = self.SwapMoveSolutions[0]

        return bestNeighborhoodSolution

    def Update(self, permutation):
        self.Permutation = permutation

        self.Moves.clear()
        self.SwapMoveSolutions.clear() # clear用于清空列表等容器

    def LocalSearch(self, neighborhoodEvaluationStrategy, solution):
        hasSolutionImproved = True

        while hasSolutionImproved:
            self.Update(solution.Permutation)
            self.DiscoverMoves()
            self.EvaluateMoves(neighborhoodEvaluationStrategy)

            bestNeighborhoodSolution = self.MakeBestMove()

            if bestNeighborhoodSolution.Makespan < solution.Makespan:
                print("New best solution has been found!")
                print(bestNeighborhoodSolution)

                self.SolutionPool.AddSolution(bestNeighborhoodSolution)

                solution.Permutation = bestNeighborhoodSolution.Permutation
                solution.Makespan = bestNeighborhoodSolution.Makespan
            else:
                print(f"Reached local optimum of {self.Type} neighborhood. Stop local search.")
                hasSolutionImproved = False       


---
## 4. Ablauflogik der Verbesserungsverfahren
---

Wir lagern die Logik in das Modul `Neighborhood.py` aus und betten die Nachbarschaften in eine Klasse `ImprovementAlgorithm` ein, welche die Logik des Verbesserungsverfahren steuert.

In [43]:
""" Base class for several types of improvement algorithms. """ 
class ImprovementAlgorithm:
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        self.InputData = inputData  # 实际有两种neighborhood，另一种是Insertion，这里只是简单初始化

        self.EvaluationLogic = None
        self.SolutionPool = None
        self.RNG = None # 随机数

        self.NeighborhoodEvaluationStrategy = neighborhoodEvaluationStrategy
        self.NeighborhoodTypes = neighborhoodTypes
        self.Neighborhoods = {} # 数据类型是字典！！！！

    def Initialize(self, evaluationLogic, solutionPool, rng = None):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

    def InitializeNeighborhoods(self, solution): # 字典Neighborhood中，可能有两个键Insertion和Swap，添加新数据时，需要用到相应的构造器
        self.Neighborhoods['Swap'] = SwapNeighborhood(self.InputData, solution.Permutation, self.EvaluationLogic, self.SolutionPool)
        # add further neighborhoods 详见class ImprovementAlgorithm文件

Die Klasse `ImprovementAlgorithm` enthält nun alle grundlegenden Komponenten für nachbarschaftsbasierte Verbesserungsverfahren. Diese können in verschiedenen Algorithmen verwendet werden. Hier erstellen wir eine Klasse `IterativeImprovement`, bei der die Lösung iterativ durch eine (sequentielle) lokale Suche verbessert wird.

In [44]:
""" Iterative improvement algorithm through sequential variable neighborhood descent. """
class IterativeImprovement(ImprovementAlgorithm): # einzelne Nachbaren
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        super().__init__(inputData, neighborhoodEvaluationStrategy, neighborhoodTypes)

    def Run(self, solution): #startSolution ,老师打错了吗？
        self.InitializeNeighborhoods(solution)  # 初始化即构造对象  

        # According to "Hansen et al. (2017): Variable neighorhood search", this is equivalent to the 
        # sequential variable neighborhood descent with a pipe neighborhood change step.
        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.Neighborhoods[neighborhoodType]

            neighborhood.LocalSearch(self.NeighborhoodEvaluationStrategy, solution)
        
        return solution

Die Klasse `Solver` wird um die Methoden `ImprovementPhase()` und `RunLocalSearch()` erweitert.

In [45]:
class Solver:
    def __init__(self, inputData, seed):
        
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)  #随机数生成器

        self.EvaluationLogic = EvaluationLogic(inputData) # 定语是El对象，可以调用类中的方法
        self.SolutionPool = SolutionPool()
        
        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool)

    def ConstructionPhase(self, constructiveSolutionMethod):
        self.ConstructiveHeuristic.Run(self.InputData, constructiveSolutionMethod)

        bestInitalSolution = self.SolutionPool.GetLowestMakespanSolution()

        print("Constructive solution found.")
        print(bestInitalSolution)

        return bestInitalSolution


    def ImprovementPhase(self, startSolution, algorithm):
        algorithm.Initialize(self.EvaluationLogic, self.SolutionPool, self.RNG)
        bestSolution = algorithm.Run(startSolution)

        print("Best found Solution.")
        print(bestSolution)

    def RunLocalSearch(self, constructiveSolutionMethod, algorithm):
        startSolution = self.ConstructionPhase(constructiveSolutionMethod)

        self.ImprovementPhase(startSolution, algorithm)

In [46]:
from Solver import *

data = InputData('InputFlowshopSIST.json')

solver = Solver(data, 2048)

improvementAlgorithm = IterativeImprovement(data)


solver.RunLocalSearch('FCFS', improvementAlgorithm)
solver.RunLocalSearch('NEH', improvementAlgorithm)


Generating an initial solution according to FCFS.
Constructive solution found.
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The permutation [0, 1, 2, 7, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 8381
The permutation [7, 1, 2, 0, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 7954
The permutation [7, 10, 2, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7472
The permutation [7, 2, 10, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7184
The permutation [7, 2, 10, 0, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7175
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Reached local optimum of Swap neighborhood. Stop local search.
Best found Solution.
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Generating an initial solution according to NEH.
Constructive solution found.
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Reached local optimum of Swap 

---
## 5. Wiederholung: gemeinsames Debuggen des Solvers
---

### 两种评估方式的总结

Bisher haben wir zwei Nachbarschaftbewertungsstrategien kennengelernt:
- Best Improvement (extensive Suche): alle Tausche einer Nachbarschaft werden ausgewertet. Der Tausch mit der größten Verbesserung wird ausgeführt.
- First Improvement (selektive Suche): die Tausche einer Nachbarschaft werden solange ausgewertet, bis eine Verbesserung gefunden wird. Sobald eine Verbesserung gefunden wurde, wird die Auswertung abgebrochen und der gefundene Tausch sofort durchgeführt. Bessere Tausche können somit "übersehen" werden.

#### Frage
Was denken Sie, mit welcher Bewertungsstrategie werden bessere Ergebnisse erzielt? Was sind Vor- und Nachteile der ein oder anderen Bewertungsstrategie? Begründen Sie ihre Antwort.

In [47]:
from Solver import *

data = InputData("InputFlowshopSIST.json")

constructiveSolutionMethod = 'FCFS'  #"SPT"....都可以，但是名字一定要跟函数定义中的名字一致

# 找出最优解
solverBestImprovement = Solver(data, 2048)
bestImprovementAlgorithm = IterativeImprovement(data, 'BestImprovement')
solverBestImprovement.RunLocalSearch(constructiveSolutionMethod, bestImprovementAlgorithm)

# 找出第一个优化方案
solverFirstImprovement = Solver(data, 2048)
bestImprovementAlgorithm = IterativeImprovement(data, 'FirstImprovement')
solverFirstImprovement.RunLocalSearch(constructiveSolutionMethod, bestImprovementAlgorithm)

localOptimumBestImprovement = solverBestImprovement.SolutionPool.GetLowestMakespanSolution()
localOptimumFirstImprovement = solverFirstImprovement.SolutionPool.GetLowestMakespanSolution()

print(f"Best improvement makespan: {localOptimumBestImprovement.Makespan}")
print(f"First improvement makespan: {localOptimumFirstImprovement.Makespan}")

isBestImprovementMakespanLower = localOptimumBestImprovement.Makespan < localOptimumFirstImprovement.Makespan

print(f"The best improvement strategy achieves a lower makespan: {isBestImprovementMakespanLower}")

Generating an initial solution according to FCFS.
Constructive solution found.
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The permutation [0, 1, 2, 7, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 8381
The permutation [7, 1, 2, 0, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 7954
The permutation [7, 10, 2, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7472
The permutation [7, 2, 10, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7184
The permutation [7, 2, 10, 0, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7175
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Reached local optimum of Swap neighborhood. Stop local search.
Best found Solution.
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Generating an initial solution according to FCFS.
Constructive solution found.
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The permutation [1, 0, 2, 3, 

---
## 6. Anwendung: selbständiges Programmieren
---

### Eine neue Nachbarschaft
- Überlegen Sie vorab welche Größe die Insertion Nachbarschaft hat
- Erweitern Sie den Solver um eine Insertion-Nachbarschaft
- Überprüfen Sie, ob Ihre implemtierte Nachbarschaft auch die erwartete Größe aufweist

In [48]:
from Solver import *

data = InputData("InputFlowshopSIST.json")

improvementAlgorithm = IterativeImprovement(data, 'BestImprovement', ['Swap', 'Insertion'])

solver = Solver(data, 1024)

solver.RunLocalSearch('LPT', improvementAlgorithm)

Generating an initial solution according to LPT.
Constructive solution found.
The permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649
The permutation [7, 5, 1, 6, 10, 4, 3, 0, 8, 9, 2] results in a Makespan of 9241
The permutation [7, 2, 1, 6, 10, 4, 3, 0, 8, 9, 5] results in a Makespan of 8171
The permutation [7, 2, 10, 6, 1, 4, 3, 0, 8, 9, 5] results in a Makespan of 7175
The permutation [7, 4, 10, 6, 1, 2, 3, 0, 8, 9, 5] results in a Makespan of 7057
Reached local optimum of Swap neighborhood. Stop local search.


IndexError: list index out of range

In [ ]:
from Solver import *

data = InputData("InputFlowshopSIST.json")  # debug, step out的作用

improvementAlgorithm = IterativeImprovement(data, 'BestImprovement', ['Swap', 'Insertion'])  # 参数表要写全


solver = Solver(data, 1024)

solver.RunLocalSearch('LPT', improvementAlgorithm)

Generating an initial solution according to LPT.
Constructive solution found.
The permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649
The permutation [7, 5, 1, 6, 10, 4, 3, 0, 8, 9, 2] results in a Makespan of 9241
The permutation [7, 2, 1, 6, 10, 4, 3, 0, 8, 9, 5] results in a Makespan of 8171
The permutation [7, 2, 10, 6, 1, 4, 3, 0, 8, 9, 5] results in a Makespan of 7175
The permutation [7, 4, 10, 6, 1, 2, 3, 0, 8, 9, 5] results in a Makespan of 7057
Reached local optimum of Swap neighborhood. Stop local search.


IndexError: list index out of range

In [ ]:
from Solver import *
from ImprovementAlgorithm import *

data = InputData("InputFlowshopSIST.JSON")

solver = Solver(data, 2048)

improvementAlgorithm = IterativeImprovement(data)

solver.RunLocalSearch('FCFS', improvementAlgorithm)

Generating an initial solution according to FCFS.
Constructive solution found.
The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The permutation [0, 1, 2, 7, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 8381
The permutation [7, 1, 2, 0, 4, 5, 6, 3, 8, 9, 10] results in a Makespan of 7954
The permutation [7, 10, 2, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7472
The permutation [7, 2, 10, 0, 4, 5, 6, 3, 8, 9, 1] results in a Makespan of 7184
The permutation [7, 2, 10, 0, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7175
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038
Reached local optimum of Swap neighborhood. Stop local search.
Best found Solution.
The permutation [7, 2, 0, 10, 4, 1, 6, 3, 8, 9, 5] results in a Makespan of 7038


### Zusatz 
Nun existieren mehrere Nachbarschaften. Überlegen Sie wie man einen sinnvollen Wechsel zwischen den Nachbarschaften steuern kann. Momentan ist in der Intesivierungsphase ein Sequential Variable Neighborhood Descent mit einem Pipe Neighborhood Change Step implementiert. Vollziehen Sie die Richtigkeit dieser Aussage nach.

- Implementieren Sie einen der anderen vorgestellten Neighborhood Change Steps: "Basic"
- Welche Vor- und Nachteile erwarten Sie dadurch?
- Welche Schwierigkeit erwarten Sie bei der Umsetzung eines "Cyclic" Neighborhood Change Steps?